In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import re
import gc
from collections import Counter

from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

In [ ]:
drive.mount('./drive')

Mounted at ./drive


In [ ]:
dataset_dir = '/content/drive/MyDrive/life/'

# train = pd.read_csv(dataset_dir+'train.csv')
# test = pd.read_csv(dataset_dir+'test.csv')
sample_submission = pd.read_csv(dataset_dir+'sample_submission.csv')
train = pd.read_csv(dataset_dir + 'train_middle_course1.csv')
test = pd.read_csv(dataset_dir + 'test_middle_course1.csv')

# EDA

In [ ]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [ ]:
# gene_names = train.columns[2:]

# for gene_name in gene_names:
#   values = train[gene_name].values
#   lowercase_values = {value for value in values if re.search(r'[a-z]', str(value)) and 'fs' not in str(value)}

#   if lowercase_values:
#     print(f'Gene: {gene_name}, Values with lowercase: {lowercase_values}')

In [ ]:
gene_names = test.columns[1:]

for gene_name in gene_names:
  values = test[gene_name].values
  lowercase_values = {value for value in values if re.search(r'[a-z]', str(value)) and 'fs' not in str(value)}

  if lowercase_values:
    print(f'Gene: {gene_name}, Values with lowercase: {lowercase_values}')

Gene: ABCA1, Values with lowercase: {'R20del'}
Gene: ABCA6, Values with lowercase: {'L218del'}
Gene: ABCB6, Values with lowercase: {'N162_Q172del'}
Gene: ABCC1, Values with lowercase: {'C1479_T1480insFND', 'K1272del'}
Gene: ABCC4, Values with lowercase: {'E231del E306del'}
Gene: ABCE1, Values with lowercase: {'E341del'}
Gene: ABCG2, Values with lowercase: {'K360del K358del'}
Gene: ABI3BP, Values with lowercase: {'G1343_T1344insFLM G1398_T1399insFLM G972_T973insFLM G721_T722insFLM G696_T697insFLM G1400_T1401insFLM G1273_T1274insFLM G1414_T1415insFLM G850_T851insFLM G1165_T1166insFLM', 'T706del T763del T656del T753del T777del T682del'}
Gene: ABL1, Values with lowercase: {'K609del K628del', 'E355del E374del'}
Gene: ABR, Values with lowercase: {'G121del G293del G302del G339del 292_293del 597_598del'}
Gene: ACADS, Values with lowercase: {'K335_D336insK K331_D332insK'}
Gene: ACADVL, Values with lowercase: {'K322del K223del K299del K277del'}
Gene: ACAT2, Values with lowercase: {'K266del K295d

del

R649del : R649del은 649번째 아미노산인 R이 유전자에서 삭제된 변이

ins

C1479_T1480insFND : 유전자 서열에서 1479번째와 1480번째 위치의 두개의 아미노산 CT에 새로운 세 개의 아미노산 FND가 삽입되었다.


**Test 데이터에는 새로운 문자 ins가 포함되어있음.**

In [ ]:
gene_names = train.columns[2:]

for gene_name in gene_names:
  values = train[gene_name].values
  values_len = len([value for value in values if value != 'WT'])
  if values_len == 1:
    print([value for value in values if value != 'WT'])

['F101fs']
['K43K']
['A19P']
['R62C']
['A70A']


In [ ]:
wt_all = train.loc[(train[train.columns[2:]] == 'WT').all(axis = 1), 'SUBCLASS']
wt_not_all = train.loc[~(train[train.columns[2:]] == 'WT').all(axis = 1), 'SUBCLASS']

# 그레이 박스 접근
# train['IS_WT'] = (train[train.columns[2:]] == 'WT').all(axis=1).astype(int)

In [ ]:
wt_all_ratio = wt_all.value_counts() / len(wt_all)
wt_not_all_ratio = wt_not_all.value_counts() / len(wt_not_all)

In [ ]:
wt_all_ratio / wt_not_all_ratio # CESC, PCPG, LAML, THCA, THYM (count ratio > 1)

,count
SUBCLASS,
ACC,NaN
BLCA,NaN
BRCA,0.415929
CESC,1.282265
COAD,NaN
DLBC,NaN
GBMLGG,0.283085
HNSC,NaN
KIPAN,0.508556


In [ ]:
train = train.fillna('WT')
test = test.fillna('WT')

In [ ]:
def find_all_wt_columns(df):
  wt_columns = [col for col in df.columns if (df[col] == 'WT').all()]
  return wt_columns

wt_columns = find_all_wt_columns(train)

In [ ]:
def find_all_wt_rows(df):
  wt_rows = df[(df == 'WT').all(axis=1)]
  return wt_rows

wt_rows = find_all_wt_rows(train)

In [ ]:
train = train.drop(wt_columns, axis = 1)
test = test.drop(wt_columns, axis = 1)

In [ ]:
# def shorten_new_variation(variation):
#     return re.sub(r'ins\s+(\w{1,4})\w*', r'ins \1', variation)

def remove_new_variation(variation):
  # 'ins'와 그 다음 문자열을 모두 제거
  return re.sub(r'ins\w*.*', '', variation)

for gene in train.columns[2:]:
    train[gene] = train[gene].apply(lambda x: remove_new_variation(x))

for gene in test.columns[1:]:
    test[gene] = test[gene].apply(lambda x: remove_new_variation(x))

데이터프레임의 열 헤더는 (A2M~ZYX) 4384개의 유전자명으로 구성되어 있음.
  - WT(Wild Type): 변이 없음 변이가 없는 원래의 상태, 단백질 자연 상태
  - {알파벳}{숫자}{알파벳 또는 *}: 단백질 서열(시퀸스), 변이를 나타냄.
  - {숫자}번째 위치의 아미노산 {알파벳}이 {알파벳}으로 변이되었음을 의미.
  - '*' 인 경우: 해당 위치에서 {알파벳}이 결실(삭제)되었음을 의미.
  - 'fs'인 경우: 프레임 쉬프트(frameshift), 완전히 다른 구조적 변화가 발생한 경우 즉 돌연변이.

  이는 단백질 기능 및 유전자 발현에 큰 영향을 미칠 수 있음

  - ex) V1255V V814V V551M 이런 구조는 변하지 않은 변이, 동의성 변이 (단백질의 구조나 기능에는 직접적인 영향을 미치지 않음)

  따라서 V1255V와 V814V는 동의성 변이이며, V551M 비동의성 변이 (551 위치에서 발린이 메티오닌으로 변이)
  
  - '>'는 대체 되었음을 의미
  - {숫자}_{숫자} 는 {숫자}번째 및 {숫자}번째에서



  - ex) * 682 *:
  - ex) 49_50WE>*K: 49번째 및 50번째 아미노산 위치에서 대체
  - ex) Q636* M539I E287K: 변이가 두번 나타난 것으로 보임
  - ex) 402_403ED>DY: 402번째 및 403번째 위치에서 ED에서 '>' DY로 대체, 변환되었음을 의미.

---

  value_counts() 즉 빈도 분석을 해본 결과 -> 위치 정보는 불필요

In [ ]:
# 위치 정보 추출 및 빈도 기반 분석

def extract_position_info(data, column):
  position_infos = []
  acid_infos = []
  for sequence in data[column]:
    position_info = re.findall(r'\d+', sequence)
    if position_info:
      position_infos.append(position_info)
    else:
      position_infos.append(None)

    position_info_cleaned = re.sub(r'\d+', '', sequence)
    acid_infos.append(position_info_cleaned)

  return position_infos, acid_infos

In [ ]:
position_infos, acid_infos = extract_position_info(train, 'A2M')
# position_infos, acid_infos = extract_position_info(test, 'A2M')

In [ ]:
# Counter([pos for sublist in position_infos if sublist is not None for pos in sublist])
# 빈도 기반 분석 결과 : 위치 정보는 불필요하다고 판단 따라서 위치 정보 제거!

In [ ]:
# 위치 정보 제거

def clear_position_info(data, st_row = 2):
  for gene_name in data.columns[st_row:]:
    # 위치 정보에 포함되는 언더바 _ 문자도 함께 제거

    data[gene_name] = data[gene_name].apply(lambda x: re.sub(r'[\d_]+', '>', x))

  return data

In [ ]:
train = clear_position_info(train)
test = clear_position_info(test, st_row = 1)

처음부터 > 변이 표시가 나오는건 일반적인 형식이 아님 따라서 제거함.

* 종결 코돈 뒤에 아미노산이 나올 순 없음 따라서 제거함. 혹은 *를 제거하고 아미노산을 남김.

ex) >WE> * K --> WE> *

In [ ]:
# 일반적인 포멧 형식

# def convert_format(data):
#   data_ = data.copy()
#   for gene_name in data_.columns[2:]:
#     sequences = data_[gene_name].str.split(' ')

#     cleaned_sequences = []
#     for sequence in sequences:
#       cleaned_sequence = [seq.lstrip('>') for seq in sequence]
#       cleaned_sequence = [re.sub('\*.*', '*', cleaned_seq) for cleaned_seq in cleaned_sequence]

#       cleaned_sequences.append(' '.join(cleaned_sequence))

#     data_[gene_name] = cleaned_sequences

#   return data_

def clean_sequence(sequence):
  sequence = sequence.lstrip('>')

  sequence = re.sub(r'\*(\w{1,})', '*', sequence) # '*' 뒤에 아미노산이 있을 경우 그 아미노산을 제거
  # sequence = re.sub(r'\*(?=\w)', '', sequence) # 전방 탐색, '*' 뒤에 아미노산이 있을 경우 '*'을 제거

  return sequence

def convert_format(data, st_row = 2):
  for gene_name in data.columns[st_row:]:
    data[gene_name] = data[gene_name].str.split(' ').apply(
        lambda sequences: ' '.join(clean_sequence(sequence) for sequence in sequences)
    )

  return data

In [ ]:
train = convert_format(train)
test = convert_format(test, st_row = 1)

동의성 변이 처리

1. 동의성 변이를 제거함.
2. 동의성 변이를 그냥 둠.

In [ ]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R>R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T>S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [ ]:
# 동의성 변이 처리

def find_synonymy(sequence):
  if '>' in sequence:
    parts = sequence.split('>')
    if len(parts) == 2:
      before, after = parts
      if before == after:
        return 'sv' # synonymous variation
    elif len(parts) == 3:
      sequence
  return sequence

def process_synonymy(data, st_row = 2):
  for gene_name in data.columns[st_row:]:
    data[gene_name] = data[gene_name].str.split(' ').apply(
        lambda sequences: ' '.join(find_synonymy(sequence) for sequence in sequences)
    )

  return data

In [ ]:
train = process_synonymy(train)
test = process_synonymy(test, st_row = 1)

In [ ]:
# 중복 변이 처리

def find_duplication(sequence):
  unique_sequence = list(set(sequence))
  return ' '.join(unique_sequence)

def process_duplication(data, st_row = 2):
  for gene_name in data.columns[st_row:]:
    data[gene_name] = data[gene_name].str.split(' ').apply(
        lambda sequence: find_duplication(sequence)
    )

    data[gene_name] = data[gene_name].str.lstrip()
    data[gene_name] = data[gene_name].str.rstrip()

  return data

In [ ]:
data = {'', 'P>T', 'V>M V>M', 'L>S', 'S>F S>L P>L S>F S>L P>L', 'A>fs S>fs', 'S>R'}
df = pd.DataFrame(data)

In [ ]:
train = process_duplication(train)
test = process_duplication(test, st_row = 1)

In [ ]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,sv,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [ ]:
train.to_csv('train_middle_course1.csv', index = False)
test.to_csv('test_middle_course1.csv', index = False)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

In [ ]:
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

In [ ]:
labels = torch.tensor(train['SUBCLASS'].values, dtype=torch.long)

In [ ]:
labels

tensor([ 8, 19, 20,  ...,  4, 22, 20])

In [ ]:
one_hot_encoder = OneHotEncoder(sparse_output = False) # sparse = False
labels_one_hot = one_hot_encoder.fit_transform(labels.reshape(-1, 1))

In [ ]:
labels_one_hot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
X = train.drop(columns=['SUBCLASS', 'ID'])
y_subclass = train['SUBCLASS']

In [ ]:
categorical_columns = X.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = X.copy()
X_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

In [ ]:
test_X = test.drop(columns=['ID'])
X_test_encoded = test_X.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns])

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
X_encoded.shape, X_test_encoded.shape

((6201, 4230), (2546, 4230))

In [ ]:
pca = PCA()

pca.fit(X_encoded)

explained_variance = pca.explained_variance_ratio_

cumulative_variance = np.cumsum(explained_variance)

n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1

print(f'95% 분산을 설명하는 n_components: {n_components_95}')

95% 분산을 설명하는 n_components: 696


In [ ]:
pca = PCA(n_components = n_components_95)
pca = pca.fit(X_encoded)
data_pca = pca.transform(X_encoded)
test_data_pca = pca.transform(X_test_encoded)

In [ ]:
!pip install iterative-stratification

In [ ]:
!pip install xgboost

In [ ]:
!pip install lightgbm

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import f1_score
import lightgbm as lgb

In [ ]:
import random
from sklearn.utils.validation import check_random_state
import os

In [ ]:
CFG = {
    'SEED': 42
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    check_random_state(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
val_scores = []
test_preds = []

reshaped_labels = labels_one_hot
# major_class_labels = reshaped_labels.argmax(axis=1)

splitter = MultilabelStratifiedKFold(n_splits = 5, shuffle = True, random_state = CFG['SEED'])
# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=CFG['SEED'])

for fold_idx, (train_index, val_index) in enumerate(splitter.split(data_pca, reshaped_labels)):

    print(f'Training fold {fold_idx + 1}...')

    train_data, val_data = X_encoded.values[train_index], X_encoded.values[val_index]
    train_labels, val_labels = labels[train_index], labels[val_index]

    model = XGBClassifier(n_estimators=2000, learning_rate=0.03, max_depth=20, tree_method='gpu_hist', gpu_id=0, random_state = CFG['SEED'], use_label_encoder=False, eval_metric='mlogloss')
    # model = lgb.LGBMClassifier(n_estimators = 200, learning_rate=0.01, max_depth=7, boosting_type = 'gbdt', verbose = -1, device = 'gpu', gpu_platform_id = 0, gpu_device_id = 0)
    model.fit(train_data, train_labels) # , callbacks = [lgb.early_stopping(stopping_rounds = 20)], eval_set = [(val_data, val_labels)]

    val_pred = model.predict_proba(val_data)
    test_pred = model.predict_proba(X_test_encoded.values)

    val_pred = val_pred.argmax(axis = 1)
    test_pred = test_pred.argmax(axis = 1)

    val_score = f1_score(val_labels, val_pred, average='macro')

    val_scores.append(val_score)
    test_preds.append(test_pred)

print(f'Validation F1 Scores for each fold: {val_scores}')
print(f'Validation F1 Scores for mean: {np.mean(val_scores)}')

Training fold 1...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:07:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:07:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
sample_submission['SUBCLASS'] = le_subclass.inverse_transform(test_preds[0])
sample_submission.to_csv('final_submission.csv', encoding='UTF-8-sig', index = False)

In [ ]:
sample_submission.head()

,ID,SUBCLASS
0,TEST_0000,PAAD
1,TEST_0001,KIPAN
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,STES


**변이 문자를 기준으로 다중 서열 정렬**

내 알고리즘

' > ' 변이 문자를 기준으로 (변이 전)과 (변이 후)시퀸스로 나누고

(변이 전) 시퀸스에서 가장 긴 시퀸스를 구해서 패딩을 수행 - (변이 후) 시퀸스도 동일하게 수행 -> 변이 전, 변이 후 시퀸스를 각각 다른 열에 생성

아미노산 리스트를 생성하고 정수로 인코딩한 후
+ 추가로 fs, * 특수 문자도 정수로 인코딩에 포함할 지 혹은 그대로 유지할지

임베딩 벡터로 변환

% 아니면 아예 변이 문자를 기준으로 나누지 않고 정수 인코딩 후 임베딩?


In [ ]:
# def sequence_padding(data):
#     data_ = data.copy()
#     for gene_name in data_.columns[:]:
#         max_before_seq = 0
#         max_after_seq = 0

#         # 최대 길이 계산
#         for sequences in data_[gene_name]:
#             for sequence in sequences.split(' '):
#                 if '>' in sequence:
#                     before, after = sequence.split('>')
#                     max_before_seq = max(max_before_seq, len(before))
#                     max_after_seq = max(max_after_seq, len(after))

#         # 패딩 적용
#         padded_sequences = []
#         for sequences in data_[gene_name]:
#             for sequence in sequences.split(' '):
#                 if '>' in sequence:
#                     before, after = sequence.split('>')
#                     padded_before = list(before) + ['-'] * (max_before_seq - len(before))
#                     padded_after = list(after) + ['-'] * (max_after_seq - len(after))

#                     padded_sequences.append(''.join(padded_before) + '>' + ''.join(padded_after))
#                 else:
#                     padded_before = sequence + '-' * max_before_seq
#                     padded_after = sequence + '-' * max_after_seq
#                     padded_sequences.append(padded_before + '-' + padded_after)

#             print(padded_sequences)

#         data_[gene_name] = padded_sequences

#     return data_

def multi_sequence_padding(data, st_row = 2, max_length_before=20, max_length_middle=20, max_length_after=20):
    for gene_name in data.columns[st_row:]:
        max_before_seq = 0
        max_after_seq = 0
        max_middle_seq = 0

        # Step 1: 최대 길이 계산
        for sequences in data[gene_name]:
            if (sequences == 'WT') or (sequences == 'sv'):
                continue
            for sequence in sequences.split(' '):
                if '>' in sequence:
                    parts = sequence.split('>')
                    if len(parts) == 2:
                      before, after = parts
                      middle = ''
                    elif len(parts) == 3:
                      before, middele, after = parts

                    before_length = len(before.replace('fs', '*').replace('del', '*').replace('ins', '*').replace('*', '*'))
                    middle_length = len(middle.replace('fs', '*').replace('del', '*').replace('ins', '*').replace('*', '*'))
                    after_length = len(after.replace('fs', '*').replace('del', '*').replace('ins', '*').replace('*', '*'))
                    # max_before_seq = max(max_before_seq, before_length)
                    # max_middle_seq = max(max_middle_seq, middle_length)
                    # max_after_seq = max(max_after_seq, after_length)
                    max_before_seq = min(max(max_before_seq, before_length), max_length_before)
                    max_middle_seq = min(max(max_middle_seq, middle_length), max_length_middle)
                    max_after_seq = min(max(max_after_seq, after_length), max_length_after)

        # Step 2: 패딩 적용
        padded_sequences = []
        for sequences in data[gene_name]:
            if sequences == 'WT': # 'WT' 패딩 적용 X
                padded_sequences.append('WT')
                continue
            if sequences == 'sv': # 동의성 패딩 적용 X
                padded_sequences.append('sv')
                continue
            padded_sequence = []
            for sequence in sequences.split(' '):
                if '>' in sequence:
                    parts = sequence.split('>')
                    if len(parts) == 2:  # '>'로 두 부분으로 나뉘는 경우
                        before, after = parts
                        middle = ''
                    elif len(parts) == 3:  # '>'로 세 부분으로 나뉘는 경우
                        before, middle, after = parts

                    before_length = len(before.replace('fs', '*').replace('del', '*').replace('ins', '*').replace('*', '*'))
                    middle_length = len(middle.replace('fs', '*').replace('del', '*').replace('ins', '*').replace('*', '*'))
                    after_length = len(after.replace('fs', '*').replace('del', '*').replace('ins', '*').replace('*', '*'))

                    padded_before = before + '-' * (max_before_seq - before_length)
                    padded_middle = middle + '-' * (max_middle_seq - middle_length) if middle else ''
                    padded_after = after + '-' * (max_after_seq - after_length)

                    padded_before = padded_before[:max_before_seq]
                    padded_middle = padded_middle[:max_before_seq]
                    padded_after = padded_after[:max_before_seq]

                    if middle:
                        padded_sequence.append(f'{padded_before}>{padded_middle}>{padded_after}')
                    else:
                        padded_sequence.append(f'{padded_before}>{padded_after}')

                else:
                    sequence_length = len(sequence.replace('fs', '*').replace('del', '*').replace('ins', '*').replace('*', '*'))
                    padded_sequence.append(sequence + '-' * (max_before_seq + max_after_seq - sequence_length))

            padded_sequences.append(' '.join(padded_sequence))

        data[gene_name] = padded_sequences

    return data

In [ ]:
# def multi_sequence_padding(data, st_row=2, max_length_before=20, max_length_middle=20, max_length_after=20):
#     for gene_name in data.columns[st_row:]:
#         max_before_seq = 0
#         max_after_seq = 0
#         max_middle_seq = 0

#         # Step 1: 최대 길이 계산
#         for sequences in data[gene_name]:
#             if sequences == 'WT' or sequences == 'sv':
#                 continue
#             for sequence in sequences.split(' '):
#                 if '>' in sequence:
#                     parts = sequence.split('>')
#                     if len(parts) == 2:  # Two parts
#                         before, after = parts
#                         middle = ''
#                     elif len(parts) == 3:  # Three parts
#                         before, middle, after = parts

#                     before_length = len(before.replace('fs', '*').replace('del', '*').replace('ins', '*'))
#                     middle_length = len(middle.replace('fs', '*').replace('del', '*').replace('ins', '*')) if middle else 0
#                     after_length = len(after.replace('fs', '*').replace('del', '*').replace('ins', '*'))

#                     # Update maximum lengths
#                     max_before_seq = min(max(max_before_seq, before_length), max_length_before)
#                     max_middle_seq = min(max(max_middle_seq, middle_length), max_length_middle)
#                     max_after_seq = min(max(max_after_seq, after_length), max_length_after)

#         # Step 2: 패딩 적용
#         padded_sequences = []
#         for sequences in data[gene_name]:
#             if sequences == 'WT':
#                 padded_sequences.append('WT')
#                 continue
#             if sequences == 'sv':
#                 padded_sequences.append('sv')
#                 continue
#             padded_sequence = []
#             for sequence in sequences.split(' '):
#                 if '>' in sequence:
#                     parts = sequence.split('>')
#                     if len(parts) == 2:
#                         before, after = parts
#                         middle = ''
#                     elif len(parts) == 3:
#                         before, middle, after = parts

#                     before_length = len(before.replace('fs', '*').replace('del', '*').replace('ins', '*'))
#                     middle_length = len(middle.replace('fs', '*').replace('del', '*').replace('ins', '*')) if middle else 0
#                     after_length = len(after.replace('fs', '*').replace('del', '*').replace('ins', '*'))

#                     # Create padded sections
#                     padded_before = before + '-' * (max_before_seq - before_length)
#                     padded_middle = (middle + '-' * (max_middle_seq - middle_length)) if middle else ''
#                     padded_after = after + '-' * (max_after_seq - after_length)

#                     # Truncate to maximum lengths
#                     padded_before = padded_before[:max_before_seq]
#                     padded_middle = padded_middle[:max_middle_seq]
#                     padded_after = padded_after[:max_after_seq]

#                     # Append padded sequence
#                     if middle:
#                         padded_sequence.append(f'{padded_before}>{padded_middle}>{padded_after}')
#                     else:
#                         padded_sequence.append(f'{padded_before}>{padded_after}')

#                 else:
#                     sequence_length = len(sequence.replace('fs', '*').replace('del', '*').replace('ins', '*'))
#                     padded_sequence.append(sequence + '-' * (max_before_seq + max_after_seq - sequence_length))

#             padded_sequences.append(' '.join(padded_sequence))

#         data[gene_name] = padded_sequences

#     return data

In [ ]:
# data = {'WT', 'P>T', 'V>M', 'L>S S>T', 'S>fs', 'SA>RA', ''}
# df = pd.DataFrame(data)

In [ ]:
# df

In [ ]:
# sequence_padding(df)

In [ ]:
# df_padded = multi_sequence_padding(df)
# df_padded

In [ ]:
train = train.drop(['AHNAK', 'DST', 'FBN1'], axis = 1)
test = test.drop(['AHNAK', 'DST', 'FBN1'], axis = 1)

In [ ]:
train = multi_sequence_padding(train)
test = multi_sequence_padding(test, st_row = 1)

In [ ]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,sv,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [ ]:
# gene_names = train.columns[2:]

# for gene_name in gene_names:
#   print(set(train[gene_name].values))

In [ ]:
# gene_names = test.columns[1:]

# for gene_name in gene_names:
#   print(set(test[gene_name].values))

In [ ]:
# '*', '-', 'fs', 'del', '>', 'WT'

In [ ]:
for gene_name in train.columns[2:]:
  train[gene_name] = train[gene_name].apply(
      lambda x: 'wt' if 'WT' in x else x
  )

for gene_name in test.columns[1:]:
  test[gene_name] = test[gene_name].apply(
      lambda x: 'wt' if 'WT' in x else x
  )

In [ ]:
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

special_symbols = {
    'X': 21,
    '*': 21,
    'fs': 22,
    'del': 23,
    'wt': 24,
    '-': 25,
    '>': 26,
    'sv': 27
}

encoding_dict = {symbol: idx for idx, symbol in enumerate(special_symbols)}

for idx, aa in enumerate(amino_acids, start = len(special_symbols)+1):
  encoding_dict[aa] = idx-1

encoding_dict[' '] = 0 # padding value
encoding_dict['X'] = 1 # 종결 코돈

In [ ]:
encoding_dict

{'X': 1,
 '*': 1,
 'fs': 2,
 'del': 3,
 'wt': 4,
 '-': 5,
 '>': 6,
 'sv': 7,
 'A': 8,
 'C': 9,
 'D': 10,
 'E': 11,
 'F': 12,
 'G': 13,
 'H': 14,
 'I': 15,
 'K': 16,
 'L': 17,
 'M': 18,
 'N': 19,
 'P': 20,
 'Q': 21,
 'R': 22,
 'S': 23,
 'T': 24,
 'V': 25,
 'W': 26,
 'Y': 27,
 ' ': 0}

In [ ]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,wt,wt,wt,wt,wt,wt,wt,wt,...,wt,wt,wt,wt,wt,wt,wt,wt,wt,wt
1,TRAIN_0001,SARC,wt,wt,wt,wt,wt,wt,wt,wt,...,wt,wt,wt,wt,wt,wt,wt,wt,wt,wt
2,TRAIN_0002,SKCM,sv,wt,wt,wt,wt,wt,wt,wt,...,wt,wt,wt,wt,wt,wt,wt,wt,wt,wt
3,TRAIN_0003,KIRC,wt,wt,wt,wt,wt,wt,wt,wt,...,wt,wt,wt,wt,wt,wt,wt,wt,wt,wt
4,TRAIN_0004,GBMLGG,wt,wt,wt,wt,wt,wt,wt,wt,...,wt,wt,wt,wt,wt,wt,wt,wt,wt,wt


In [ ]:
# def encode_sequence(data, encode_dict):
#   data_ = data.copy()
#   for gene_name in data[2:]:
#     data_[gene_name] = data_[gene_name].apply(
#         lambda x: ''.join(ㄷ
#             str(encode_dict.get(token, token))
#             for token in x
#         )
#     )

#   return data_

def encode_sequence(data, encode_dict, st_row = 2):
    # 두 글자 이상의 기호들을 먼저 처리하기 위해 정렬함.
    multiple_tokens = sorted([token for token in encode_dict.keys() if len(token) > 1], key=len, reverse=True)

    for gene_name in data.columns[st_row:]:
        data[gene_name] = data[gene_name].apply(lambda x: encode_tokens(x, encode_dict, multiple_tokens))

    return data

def encode_tokens(seq, encode_dict, multiple_tokens):
    # 두 글자 이상의 기호들을 먼저 인코딩
    for token in multiple_tokens:
        seq = seq.replace(token, str(encode_dict[token]))

    # 그 후 나머지 한 글자 기호들을 인코딩
    encoded_seq = [str(encode_dict.get(char, 0)) for char in seq]
    # encoded_seq = ''.join(str(encode_dict.get(char, char)) for char in seq)

    return [int(encoded) for encoded in encoded_seq if encoded.isdigit()] # 문자열이 숫자로만 구성되어 있는지 확인

In [ ]:
train = encode_sequence(train, encoding_dict)
test = encode_sequence(test, encoding_dict, st_row = 1)

In [ ]:
len(set(encoding_dict.values()))
# 28 why? >> 종결 코돈 'X', '*' -> 1

28

In [ ]:
# gene_names = train_.columns[2:]

# # for gene_name in gene_names:
# #   print(set(train_[gene_name].values))

# for gene_name in gene_names:
#   print(set(tuple(seq) for seq in train_[gene_name].values))

In [ ]:
# for gene_name in train_.columns[2:]:
#   train_[gene_name] = [list(map(int, seq.split())) for seq in train_[gene_name]]

In [ ]:
# train_.head()

In [ ]:
# 임베딩 병합, 임베딩 합치기

# 메모리 초과시 데이터프레임 전처리 과정을 저장하고 불러오기 방향

In [ ]:
# vocab_size = max(encoding_dict.values()) + 1
# embedding_dim = 50

# embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = 0)
# embedding = embedding.to(device)

# for gene_name in train_.columns[2:]:
#   sequences = [torch.tensor(seq, dtype = torch.long).to(device)
#                 for seq in train_[gene_name].values]

#   padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)

#   embedded_sequences = embedding(padded_sequences)

#   train_[gene_name] = [embedded_seq.detach().numpy() for embedded_seq in embedded_sequences]

# vocab_size = max(encoding_dict.values()) + 1
# embedding_dim = 10 # 50

# embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = 0)
# embedding = embedding.to(device)

# batch_size = 1000

# for gene_name in train_.columns[2:]:
#   sequences = [torch.tensor(seq, dtype = torch.long) for seq in train[gene_name].values]
#   embedded_batched = []

#   for i in range(0, len(sequences), batch_size):
#     batch_sequences = sequences[i:i+batch_size]
#     padded_sequences = pad_sequence(batch_sequences, batch_first = True, padding_value = 0).to(device)
#     embedded_sequences = embedding(padded_sequences)
#     embedded_batched.extend([embedded_seq.detach().cpu().numpy() for embedded_seq in embedded_sequences])

#   train_[gene_name] = embedded_batched

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
# encoder = LabelEncoder()
# train['SUBCLASS'] = encoder.fit_transform(train['SUBCLASS'].values)

In [ ]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
1,TRAIN_0001,SARC,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
2,TRAIN_0002,SKCM,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
3,TRAIN_0003,KIRC,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
4,TRAIN_0004,GBMLGG,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
6197,TRAIN_6197,LGG,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
6198,TRAIN_6198,COAD,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],"[24, 6, 23]",[0]
6199,TRAIN_6199,TGCT,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]


In [ ]:
max_length_local = 0

for gene_name in train.columns[2:]:
  max_length = max(len(seq) for seq in train[gene_name])

  if max_length > max_length_local:
    max_length_local = max_length

print("Maximum sequence length:", max_length_local)

Maximum sequence length: 95


In [ ]:
# max_length = 0
# for gene_name in train.columns[2:]:
#   sequences = [torch.tensor(seq, dtype = torch.long) for seq in train[gene_name].values]
#   max_length = max(max_length, max(len(seq) for seq in sequences))

In [ ]:
max_length_local = 0

for gene_name in test.columns[1:]:
    max_length = max(len(seq) for seq in test[gene_name])

    if max_length > max_length_local:
        max_length_local = max_length

print("Maximum sequence length:", max_length_local)

Maximum sequence length: 77


In [ ]:
# max_length = 0
# for gene_name in test.columns[1:]:
#     sequences = [torch.tensor(seq, dtype=torch.long) for seq in test[gene_name].values]
#     max_length = max(max_length, max(len(seq) for seq in sequences))

# print("Maximum length of tensor sequences:", max_length)

In [ ]:
# train_padded_sequences = []

# for gene_name in train_.columns[2:]:
#   sequences = [torch.tensor(seq) for seq in train_[gene_name].values]
#   padded_sequences = pad_sequence(sequences, batch_first = True, padding_value = 0)

#   if padded_sequences.size(1) < max_length:
#     padded_sequences = torch.nn.functional.pad(padded_sequences, (0, max_length - padded_sequences.size(1)), value = 0)

#   train_padded_sequences.append(padded_sequences)

In [ ]:
train.shape

(6201, 4229)

In [ ]:
test.shape

(2546, 4228)

In [ ]:
train.to_csv('train_middle_course.csv', index = False)

In [ ]:
test.to_csv('test_middle_course.csv', index = False)

In [ ]:
def pad_sequence_func(sequences, max_length):
  padded_sequences = torch.zeros((len(sequences), max_length))

  for i, seq in enumerate(sequences):
    length = min(len(seq), max_length)
    padded_sequences[i, :length] = seq[:length].detach().clone().to(torch.float32)

  return padded_sequences

In [ ]:
import ast

def parse_sequence(seq):
    try:
        return ast.literal_eval(seq)
    except (ValueError, SyntaxError):
        return None

In [ ]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
1,TRAIN_0001,SARC,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
2,TRAIN_0002,SKCM,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
3,TRAIN_0003,KIRC,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
4,TRAIN_0004,GBMLGG,[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]


In [ ]:
test.head()

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TEST_0000,[0],[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
1,TEST_0001,[0],[0],[0],[0],"[22, 6, 21]",[0],[0],[0],[0],...,[0],[0],[0],[0],[0],"[15, 6, 23]",[0],[0],[0],[0]
2,TEST_0002,[0],[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
3,TEST_0003,[0],[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]
4,TEST_0004,[0],[0],[0],[0],[0],[0],[0],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]


In [ ]:
4230/4

1057.5

In [ ]:
# max_length = 162
# train_padded_sequences = []

# for gene_name in train.columns[2:1096]:
#     sequences = [parse_sequence(seq) for seq in train[gene_name].values]
#     sequences = [torch.tensor(seq, dtype=torch.float32, device=device) for seq in sequences if seq is not None and isinstance(seq, list)]

#     with torch.no_grad():
#         padded_sequences = pad_sequence_func(sequences, max_length, device)

#     train_padded_sequences.append(padded_sequences)

#     del sequences, padded_sequences
#     torch.cuda.empty_cache()
#     gc.collect()

# # (6201, 162, 1098)
# train_padded_sequences = torch.stack(train_padded_sequences, dim=-1)

# torch.save(train_padded_sequences, 'train_padded_sequences1.pt')

In [ ]:
test.shape

(2546, 4228)

In [ ]:
4228 / 4

1057.0

In [ ]:
max_length = 95
train_padded_sequences = []

for gene_name in train.columns[1057*3:]:
    sequences = [parse_sequence(seq) for seq in train[gene_name].values]
    sequences = [torch.tensor(seq, dtype=torch.float32) for seq in sequences if seq is not None and isinstance(seq, list)]

    with torch.no_grad():
        padded_sequences = pad_sequence_func(sequences, max_length)

    train_padded_sequences.append(padded_sequences.unsqueeze(1))

    del sequences, padded_sequences
    # torch.cuda.empty_cache()
    gc.collect()

train_padded_sequences = torch.cat(train_padded_sequences, dim=1)

torch.save(train_padded_sequences, 'train_padded_sequences4.pt')

In [ ]:
train_padded_sequences.shape

torch.Size([6201, 1058, 95])

In [ ]:
train_padded_sequences[0].shape

torch.Size([1058, 95])

In [ ]:
del train_padded_sequences

# torch.cuda.empty_cache
gc.collect()

0

In [ ]:
max_length = 77
test_padded_sequences = []

for gene_name in test.columns[1057*3:]:
    sequences = [parse_sequence(seq) for seq in test[gene_name].values]
    sequences = [torch.tensor(seq, dtype=torch.float32) for seq in sequences if seq is not None and isinstance(seq, list)]

    with torch.no_grad():
        padded_sequences = pad_sequence_func(sequences, max_length)

    test_padded_sequences.append(padded_sequences.unsqueeze(1))

    del sequences, padded_sequences
    # torch.cuda.empty_cache()
    gc.collect()

test_padded_sequences = torch.cat(test_padded_sequences, dim=1)

torch.save(test_padded_sequences, 'test_padded_sequences4.pt')

In [ ]:
from google.colab import files
files.download('/content/test_padded_sequences3.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/test_padded_sequences4.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_padded_sequences.shape

torch.Size([2546, 1057, 77])

In [ ]:
test_padded_sequences[0].shape

torch.Size([1057, 77])

In [ ]:
del test_padded_sequences

# torch.cuda.empty_cache
gc.collect()

0

In [ ]:
import gc

In [ ]:
parts = []
for i in range(4):
  part = torch.load(f'/content/train_padded_sequences{i+1}.pt')
  parts.append(part)
  del part
  gc.collect()

In [ ]:
train_padded_sequences = torch.cat((part1, part2, part3, part4), dim = 1)

# del part1, part2, part3, part4
# torch.cuda.empty_cache()
# gc.collect()

torch.save(train_padded_sequences, 'train_padded_sequences.pt')

<ipython-input-4-b3184a21ffe4>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  part1 = torch.load('/content/train_padded_sequences1.pt')
<ipython-input-4-b3184a21ffe4>:2: F

In [ ]:
train_padded_sequences

In [ ]:
train_labels = torch.tensor(train['SUBCLASS'].values, dtype=torch.long)

In [ ]:
torch.save({
    'padded_sequences': train_padded_sequences,
    'labels': train_labels
}, 'train_data.pth')

In [ ]:
train_data = torch.load('train_data.pth')
train_padded_sequences = train_data['padded_sequences']
train_labels = train_data['labels']

In [ ]:
train_dataset = TensorDataset(train_padded_sequences, train_labels)

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)